<!--BOOK_INFORMATION-->
<img style="float: right; width: 100px" src="https://raw.github.com/pyomeca/design/master/logo/logo_cropped.svg?sanitize=true">

# Effective computation in Biomechanics
## with [pyomeca](https://github.com/pyomeca/pyomeca)
Romain Martinez (martinez.staps@gmail.com | [GitHub](https://github.com/romainmartinez))

<!--NAVIGATION-->
< [Signal processing](02.02-Signal-processing.ipynb) | [Contents](Index.ipynb) | [Data visualization](03.00-Effective-viz.ipynb) >

# Other modules

Pyomeca is designed to work well with other libraries that we have developed:

## Pyosim

[Pyosim](https://github.com/pyomeca/pyosim) between OpenSim and the Pyomeca library.

It allows you to batch process multiple musculoskeletal analysis performed by the OpenSim software.

You must first generate the configuration files (`xml`) from the OpenSim GUI.

You will find a complete example in the [example folder](https://github.com/pyomeca/pyosim/tree/master/examples) of the repo.

### Create a project

```python
from pyosim import Project
from pyosim import Conf

PROJECT_PATH = Path("your_project")

project = Project(path=PROJECT_PATH)
project.create_project()

# complete `_conf.csv`

conf = Conf(project_path=PROJECT_PATH)


# add data dirs
participants = conf.get_participants_to_process()

for iparticipant in participants:
    conf.add_conf_field(
        {
            iparticipant: {
                "markers_data_dir": "path/to/your/markers/data",
                "emg_data_dir": "path/to/your/emg/data",
                "analogs_data_dir": "path/to/your/analogs/data",
            }
        }
    )
```

### Export markers

```python
from pyosim import Markers3dOsim

# loop through each trials
for iparticipant in participants:
    directories = conf.get_conf_field(
        participant=iparticipant, field=["marker_data_dir"]
    )

    for idir in directories:

        for itrial in Path(idir).glob("*.c3d"):
            Markers3dOsim.from_c3d(itrial).to_trc(
                f'{PROJECT_PATH / iparticipant / "0_markers" / itrial.stem}.trc'
            )
```

### Export EMGs

```python
from pyosim import Analogs3dOsim


# loop through each trials
for iparticipant in participants:
    directories = conf.get_conf_field(participant=iparticipant, field=["emg_data_dir"])

    for idir in directories:

        for itrial in Path(idir).glob("*.c3d"):
            (
                Analogs3d.from_c3d(ifile)
                .band_pass(freq=2000, order=4, cutoff=[10, 425])
                .center()
                .rectify()
                .low_pass(freq=2000, order=4, cutoff=5)
                .normalization(ref=mvc)
                .time_normalization()
                .to_sto(f'{PROJECT_PATH / iparticipant / "0_emg" / itrial.stem}.sto')
            )
```

### Musculoskeletal analysis

```python
from pyosim import Scale, InverseKinematics, InverseDynamics, StaticOptimization, MuscleAnalysis, JointReaction

for iparticipant in participants:
    
    Scale(...)
    
    InverseKinematics(...)
    
    InverseDynamics(...)
    
    StaticOptimization(...)
    
    MuscleAnalysis(...)
    
    JointReaction(...)
```

## Biorbd

[biorbd](https://github.com/pyomeca/biorbd) is a C++ interface and add-ons to the Rigid Body Dynamics Library, with Python and Matlab binders.

Formerly `s2mlib`, `biorbd` is still in development and not usable yet.

## Pyoviz

[pyoviz](https://github.com/pyomeca/pyoviz) is the pyomeca visualization toolkit, with some useful GUIs.

`pyoviz` is still in development and not usable yet.

<!--NAVIGATION-->
< [Signal processing](02.02-Signal-processing.ipynb) | [Contents](Index.ipynb) | [Data visualization](03.00-Effective-viz.ipynb) >